In [ ]:
#########################################################################
########------ Ciencia de Datos e IA Generativa con Python ------########
#########################################################################
# Capacitador: Julio César Bernal Fernández
# email: juliobf08@gmail.com
# Tema : Laboratorio de Series de Tiempo
# versión: 1.0
#########################################################################


# 📈 Series de Tiempo en Colab: AR, MA, ARMA, ARIMA, SARIMA y Prophet (CO2)

**Dataset real:** `CO2` (concentración atmosférica de CO₂) de `statsmodels`.  
Trabajaremos con una serie **mensual** (promedio mensual), haremos **train/test split**, entrenaremos y compararemos:
- **AR(p)**
- **MA(q)**
- **ARMA(p, q)**
- **ARIMA(p, d, q)**
- **SARIMA(p, d, q) × (P, D, Q, 12)**
- **Prophet** (Facebook/Meta)

Se evaluarán con **RMSE / MAE / MAPE**, y se incluye diagnóstico de residuos del mejor modelo.


 **AIC (Akaike Information Criterion)** es una métrica muy usada en estadística y modelos de series de tiempo para comparar modelos.

---

# 📌 Definición

El **Criterio de Información de Akaike (AIC)** mide el equilibrio entre:

1. **Bondad de ajuste** (qué tan bien el modelo explica los datos).
2. **Complejidad del modelo** (número de parámetros estimados).

Su fórmula es:

$$
AIC = 2k - 2\ln(\hat{L})
$$

donde:

* $k$ = número de parámetros del modelo.
* $\hat{L}$ = verosimilitud máxima (likelihood) del modelo.

---

# 📌 Intuición

* **Primer término (2k)**: penaliza modelos con muchos parámetros (evita sobreajuste).
* **Segundo término (-2 log L)**: recompensa modelos que se ajustan bien a los datos.

Así, el **AIC busca el modelo más parsimonioso**: el que explica bien los datos con el menor número de parámetros.

---

# 📌 Interpretación

* **Cuanto más bajo sea el AIC, mejor es el modelo** (entre modelos comparables).
* El AIC **no es un test absoluto**, solo sirve para **comparar modelos entrenados sobre los mismos datos**.
* Una diferencia de:

  * **0–2** → evidencia débil (ambos modelos son casi igual de buenos).
  * **4–7** → evidencia considerable en contra del modelo con mayor AIC.
  * **>10** → evidencia muy fuerte en contra del modelo con mayor AIC.

---

# 📌 Ejemplo práctico en series de tiempo

Cuando ajustamos **ARIMA/SARIMA**, probamos diferentes órdenes `(p,d,q)` o `(p,d,q)x(P,D,Q,s)`.

* Cada modelo tendrá un **AIC diferente**.
* Elegimos aquel con el **menor AIC**, pues balancea ajuste y simplicidad.

---

# 📌 Diferencias con BIC

El **BIC (Bayesian Information Criterion)** es similar pero **penaliza más fuerte** la complejidad (porque usa $\ln(n)$ en lugar de 2).

* AIC tiende a elegir modelos más complejos.
* BIC favorece modelos más simples (sobre todo con muestras grandes).

---

✅ En resumen:
El **AIC** es un criterio para seleccionar modelos que equilibra ajuste y parsimonia. Siempre se usa de forma **comparativa**, y el **mejor modelo es el que tiene el AIC más bajo**.




## 0) Instalación (solo Colab)

In [ ]:
!pip -q install prophet

## 1) Imports y utilidades

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error

import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox

try:
    # Prophet (Meta)
    from prophet import Prophet
except Exception as e:
    Prophet = None
    print("Prophet no disponible aún. Ejecuta la celda de instalación y reinicia el kernel si es necesario.")

# === Métricas ===
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

pd.options.display.float_format = '{:,.4f}'.format


## 2) Cargar dataset real: CO₂ (statsmodels)

- El dataset `co2` viene en `statsmodels`.
- Resampleamos a **mensual** con media y realizamos **interpolación** de faltantes.


In [ ]:
# Cargar
data = sm.datasets.co2.load_pandas().data

In [ ]:
# La columna 'co2' está indexada semanalmente; convertimos a datetime y resampleamos mensual
data = data.asfreq('W-SAT')  # garantizar frecuencia semanal en sábado
data = data['co2'].to_frame()
data.index = pd.to_datetime(data.index)

In [ ]:
# Promedio mensual y completar faltantes por interpolación
y_m = data['co2'].resample('MS').mean().interpolate('linear')
ts = y_m.rename('y')

In [ ]:
# Visualizar
print(ts.head())
ax = ts.plot(figsize=(10,4))
ax.set_title("CO₂ mensual (ppm) — promedio/interpolado")
ax.set_xlabel("Fecha")
ax.set_ylabel("ppm")
plt.show()

## 3) Train/Test split

In [ ]:
h = 36  # horizonte de test (36 meses)
train, test = ts.iloc[:-h], ts.iloc[-h:]
print("Tamaño train:", train.shape, "| test:", test.shape)


## 4) Estacionariedad (ADF) y diferencias de referencia

- La serie CO₂ tiene **tendencia** y **estacionalidad** anual.
- Para AR/MA/ARMA probamos en niveles (d=0).
- Para ARIMA y SARIMA exploramos `d=1` y `D=1` con s=12.


In [ ]:
def adf_report(series, name=""):
    stat, p, lags, nobs, *_ = adfuller(series.dropna(), autolag='AIC')
    print(f"{name} -> ADF: {stat:.4f} | p-value: {p:.4f} | lags: {lags} | nobs: {nobs}")

adf_report(train, "Train (niveles)")
adf_report(train.diff(), "Diff-1")
adf_report(train.diff(12), "Diff-12")
adf_report(train.diff().diff(12), "Diff-1 & 12")


## 5) Función auxiliar: entrenar, pronosticar y evaluar

In [ ]:
def fit_forecast_evaluate(model_name, model, steps, test_index, y_test):
    fitted = model.fit()
    fc = fitted.forecast(steps=steps)
    fc = pd.Series(fc, index=test_index, name=model_name)
    metrics = {
        "Modelo": model_name,
        "RMSE": rmse(y_test, fc),
        "MAE": mean_absolute_error(y_test, fc),
        "MAPE(%)": mape(y_test, fc),
        "AIC": getattr(fitted, "aic", np.nan),
        "BIC": getattr(fitted, "bic", np.nan)
    }
    return fitted, fc, metrics

## 6) AR(p): selección por AIC (p ∈ {1..5})

In [ ]:
best_ar, best_aic_ar, best_fit_ar = None, np.inf, None
for p in range(1, 6):
    try:
        model = ARIMA(train, order=(p,0,0))
        res = model.fit()
        if res.aic < best_aic_ar:
            best_aic_ar, best_ar, best_fit_ar = res.aic, model, res
    except Exception:
        pass

f_ar, fc_ar, m_ar = fit_forecast_evaluate(f"AR(p={best_ar.order[0]})", best_ar, h, test.index, test)
print(m_ar)

## 7) MA(q): selección por AIC (q ∈ {1..5})

In [ ]:
best_ma, best_aic_ma, best_fit_ma = None, np.inf, None
for q in range(1, 6):
    try:
        model = ARIMA(train, order=(0,0,q))
        res = model.fit()
        if res.aic < best_aic_ma:
            best_aic_ma, best_ma, best_fit_ma = res.aic, model, res
    except Exception:
        pass

f_ma, fc_ma, m_ma = fit_forecast_evaluate(f"MA(q={best_ma.order[2]})", best_ma, h, test.index, test)
print(m_ma)

## 8) ARMA(p,q): grid pequeño por AIC (p,q ∈ {0..3}, excluye (0,0))

In [ ]:
best_arma, best_aic_arma, best_fit_arma = None, np.inf, None
for p in range(0, 4):
    for q in range(0, 4):
        if p == 0 and q == 0:
            continue
        try:
            model = ARIMA(train, order=(p,0,q))
            res = model.fit()
            if res.aic < best_aic_arma:
                best_aic_arma, best_arma, best_fit_arma = res.aic, model, res
        except Exception:
            pass

f_arma, fc_arma, m_arma = fit_forecast_evaluate(f"ARMA(p={best_arma.order[0]},q={best_arma.order[2]})", best_arma, h, test.index, test)
print(m_arma)

## 9) ARIMA(p,1,q): grid por AIC (p,q ∈ {0..3})

In [ ]:
best_arima, best_aic_arima, best_fit_arima = None, np.inf, None
for p in range(0, 4):
    for q in range(0, 4):
        try:
            model = ARIMA(train, order=(p,1,q))
            res = model.fit()
            if res.aic < best_aic_arima:
                best_aic_arima, best_arima, best_fit_arima = res.aic, model, res
        except Exception:
            pass

f_arima, fc_arima, m_arima = fit_forecast_evaluate(f"ARIMA(p={best_arima.order[0]},d={best_arima.order[1]},q={best_arima.order[2]})", best_arima, h, test.index, test)
print(m_arima)

## 10) SARIMA(p,d,q)×(P,D,Q,12): parrilla por AIC

In [ ]:
p_vals, d_vals, q_vals = [0,1,2], [0,1], [0,1,2]
P_vals, D_vals, Q_vals = [0,1], [0,1], [0,1]
s = 12

best_sarima, best_aic_sarima, best_fit_sarima = None, np.inf, None
for p in p_vals:
    for d in d_vals:
        for q in q_vals:
            for P in P_vals:
                for D in D_vals:
                    for Q in Q_vals:
                        try:
                            model = SARIMAX(train, order=(p,d,q),
                                            seasonal_order=(P,D,Q,s),
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
                            res = model.fit()
                            if res.aic < best_aic_sarima:
                                best_aic_sarima, best_sarima, best_fit_sarima = res.aic, model, res
                        except Exception:
                            pass

sarima_name = f"SARIMA{best_sarima.order}x{best_sarima.seasonal_order}"
f_sarima, fc_sarima, m_sarima = fit_forecast_evaluate(sarima_name, best_sarima, h, test.index, test)
print(m_sarima)

## 11) Prophet (Facebook/Meta)

In [ ]:
if Prophet is None:
    print("Prophet no importado. Instálalo y vuelve a ejecutar esta celda.")
else:
    df_prophet = ts.reset_index().rename(columns={"index":"ds","y":"y"})
    df_train = df_prophet.iloc[:-h].copy()
    df_test  = df_prophet.iloc[-h:].copy()

    m = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='additive'  # puedes probar 'multiplicative'
    )
    m.fit(df_train)

    future = m.make_future_dataframe(periods=h, freq="MS")
    forecast = m.predict(future)

    # Extraer pronóstico para el rango de test
    fc_prophet = forecast.set_index("ds").loc[df_test["ds"], "yhat"]
    fc_prophet.name = "Prophet"

    m_prophet = {
        "Modelo": "Prophet",
        "RMSE": rmse(df_test["y"].values, fc_prophet.values),
        "MAE": mean_absolute_error(df_test["y"].values, fc_prophet.values),
        "MAPE(%)": mape(df_test["y"].values, fc_prophet.values),
        "AIC": np.nan,
        "BIC": np.nan
    }
    print(m_prophet)

    # Plot nativo de Prophet (opcional)
    # fig = m.plot(forecast); plt.show()


## 12) Comparación de métricas (Test)

In [ ]:
rows = [m_ar, m_ma, m_arma, m_arima, m_sarima]
try:
    rows.append(m_prophet)
except NameError:
    pass
results = pd.DataFrame(rows).sort_values("RMSE")
results


## 13) Gráfico: Serie real vs pronósticos (h meses)

In [ ]:
plt.figure(figsize=(11,5))
train.plot(label="Train")
test.plot(label="Test")
fc_ar.plot(label=m_ar["Modelo"])
fc_ma.plot(label=m_ma["Modelo"])
fc_arma.plot(label=m_arma["Modelo"])
fc_arima.plot(label=m_arima["Modelo"])
fc_sarima.plot(label=m_sarima["Modelo"])
try:
    fc_prophet.plot(label="Prophet")
except NameError:
    pass
plt.title("Pronósticos sobre Test")
plt.xlabel("Fecha"); plt.ylabel("CO₂ (ppm)")
plt.legend()
plt.tight_layout()
plt.show()


## 14) Diagnóstico de residuos del mejor modelo (según RMSE)

In [ ]:
best_name = results.iloc[0]["Modelo"]
fit_map = {
    m_ar["Modelo"]: f_ar,
    m_ma["Modelo"]: f_ma,
    m_arma["Modelo"]: f_arma,
    m_arima["Modelo"]: f_arima,
    m_sarima["Modelo"]: f_sarima
}
best_fit = fit_map.get(best_name, None)
if best_fit is None and best_name == "Prophet":
    print("Prophet no expone residuos del mismo modo; salta diagnóstico.")
else:
    resid = best_fit.resid
    print("Ljung–Box (lag=12):")
    print(acorr_ljungbox(pd.Series(resid).dropna(), lags=[12], return_df=True))

    fig, ax = plt.subplots(1,2, figsize=(10,4))
    ax[0].plot(resid); ax[0].set_title(f"Residuos — {best_name}")
    sm.qqplot(pd.Series(resid).dropna(), line="s", ax=ax[1])
    ax[1].set_title("QQ-plot residuos")
    plt.tight_layout(); plt.show()
